In [7]:
import h2o
h2o.init()
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,7 mins 08 secs
H2O cluster version:,3.10.4.8
H2O cluster version age:,1 month and 18 days
H2O cluster name:,H2O_from_python_root_ryu2oo
H2O cluster total nodes:,1
H2O cluster free memory:,1.551 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [ ]:
col_types = ["numeric", "numeric", "numeric", "enum", "enum", "numeric", "numeric", "numeric", "numeric"]
data = h2o.import_file("http://h2o-public-test-data.s3.amazonaws.com/smalldata/prostate/prostate.csv", col_types=col_types)
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test =  h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

#train, test = data.split_frame(ratios=[.8], seed=1)
x = ["CAPSULE","GLEASON","RACE","DPROS","DCAPS","PSA","VOL"]
y = "AGE"
nfolds = 5
my_gbm = H2OGradientBoostingEstimator(nfolds=nfolds, fold_assignment="Modulo", keep_cross_validation_predictions=True)
my_gbm.train(x=x, y=y, training_frame=train)
my_rf = H2ORandomForestEstimator(nfolds=nfolds, fold_assignment="Modulo", keep_cross_validation_predictions=True)
my_rf.train(x=x, y=y, training_frame=train)

my_glm = H2OGeneralizedLinearEstimator(nfolds=nfolds, fold_assignment="Modulo", keep_cross_validation_predictions=True)
my_glm.train(x=x, y=y, training_frame=train)

stack = H2OStackedEnsembleEstimator(model_id="my_ensemble", training_frame=train, validation_frame=test, base_models=[my_gbm.model_id, my_rf.model_id,my_glm.model_id])
stack.train(x=x, y=y, training_frame=train, validation_frame=test)

stack.model_performance()

Parse progress: |

In [17]:
my_gbm.model_performance()


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 23.9028739906
RMSE: 4.88905655424
MAE: 3.91630132575
RMSLE: 0.0753989795312
Mean Residual Deviance: 23.9028739906


In [18]:
my_rf.model_performance()


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 48.7475449815
RMSE: 6.98194421214
MAE: 5.66544878255
RMSLE: 0.106896045276
Mean Residual Deviance: 48.7475449815


In [11]:
my_glm.model_performance()


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 40.5757406849
RMSE: 6.3699090013
MAE: 5.22359545593
RMSLE: 0.0977400498009
R^2: 0.0443128952058
Mean Residual Deviance: 40.5757406849
Null degrees of freedom: 303
Residual degrees of freedom: 291
Null deviance: 12906.9703947
Residual deviance: 12335.0251682
AIC: 2016.47842004
